In [5]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

Importing the data using read_csv


In [6]:
Shakespeare = pd.read_csv("..\\data\\raw\\Shakespeare_data.csv")
Shakespeare

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
...,...,...,...,...,...,...
111391,111392,A Winters Tale,38.0,5.3.180,LEONTES,"Lead us from hence, where we may leisurely"
111392,111393,A Winters Tale,38.0,5.3.181,LEONTES,Each one demand an answer to his part
111393,111394,A Winters Tale,38.0,5.3.182,LEONTES,Perform'd in this wide gap of time since first
111394,111395,A Winters Tale,38.0,5.3.183,LEONTES,We were dissever'd: hastily lead away.


Finding the most common words in the play and creating a sparse matrix to indicate which lines contain 
which of these words. I then created columns to signify these relationships. Each column is the word and the row is a 1 if that word is  in the line and 0 otherwise. This gives us the ability to "recognize" players by the words that they speak most often. This will be helpful for characters who often speak about the same topic/person.

In [76]:
d = Shakespeare.copy()
PlayerLines = d.PlayerLine.tolist()
words =[]
lines = []
for line_words in PlayerLines:
    line_words = line_words.replace(",","")
    line_words = line_words.replace(".","")
    line_words = line_words.replace("'","")
    line_words = line_words.replace("?","")
    line_words = line_words.replace("!","")
    line_words = line_words.replace(":","")
    line_words = line_words.replace("-","")

    line_words = str.lower(line_words)
    lines.append(line_words)
    line_words  = line_words.split(" ")
    for word in line_words:
        words.append(word)
unique, count  = np.unique(words, return_counts=True)
word_counts = list(zip(unique,count  ))
word_counts.sort(key=lambda x: x[1],reverse=True)
common_words = word_counts[:500]
for word in common_words:
    arr = []
    for line in lines:
        if word[0] in line:
            arr.append(1)
        else:
            arr.append(0)
    d[word[0]] = arr

Creating a one hot encoding for the current act scene and play. We can't create a different column for each line so we are just using the play, act and scene data to make these columns. Each column is a 1 if we are in the play, act or scene and 0 otherwise.

In [78]:
tmp = d.copy()
tmp = tmp.dropna()
dummies = pd.get_dummies(tmp.Play)
tmp =tmp.drop(["Play"],axis=1)

for column in dummies.columns:
    tmp[column] = dummies[column]
ActScene = tmp.ActSceneLine.tolist()
act_scene = []
for el in ActScene:
    act_scene.append(el[:4])
tmp["act_scene"] = act_scene
dummies = pd.get_dummies(tmp.act_scene)
tmp =tmp.drop(["ActSceneLine"],axis=1)
for column in dummies.columns:
    tmp[column] = dummies[column]


Removing comlumns that are not going to be used as  our predictive 

In [89]:
X  = tmp.drop(["PlayerLinenumber","PlayerLine", "Player", "act_scene","Dataline"], axis = 1)
Y  = tmp.Player

Running the Logisitc Regression and Naive Bayes classifcation algorithms on our data. Here we see that the Logistic regression is much more predictive in this data set than the Naive Bayes. We get 74% accuracy in the Logistic Regression approach and 44% in the Naive Bayes

In [93]:

clf = LogisticRegression(random_state=0).fit(X.head(50000), Y.head(50000))

In [95]:
arr = clf.predict(X.head(50000))

pairs = list(zip(Y.head(50000).tolist(),arr))
same=0
for pair in pairs:
    if pair[0] == pair[1]:
        same += 1
print( same / len(arr))

0.74216


In [96]:
gnb = GaussianNB()
y_pred = gnb.fit(X.head(50000), Y.head(50000)).predict(X.head(50000))


In [98]:
pairs = list(zip(Y.head(50000).tolist(),y_pred))
same=0
for pair in pairs:
    if pair[0] == pair[1]:
        same += 1
print( same / len(arr))

0.44844
